# Testing Expasy query helper

Testing our Expasy query helper reusing the example queries that succeeded retrieved from `test_example_queries.ipynb`

Potential interesting questions to use for evaluation:
- Which are the human genes associated with cancer and their orthologs expressed in the rat brain?
- Select the number of proteins for each of the subclasses of ec:1.1.1.- EC (Enzyme Commission) class (from biosoda)
- 

In [1]:
import re
import os
import requests

from openai import OpenAI
from qdrant_client.models import FieldCondition, Filter, MatchValue, ScoredPoint
from rdflib.plugins.sparql import prepareQuery, prepareUpdate
from rdflib.plugins.sparql.algebra import translateQuery
from SPARQLWrapper import JSON, TURTLE, XML, SPARQLWrapper
from dotenv import load_dotenv

from expasy_chat.validate_sparql import extract_sparql_queries
from expasy_chat.config import settings

load_dotenv()
expasy_api_key = os.getenv('EXPASY_API_KEY')

example_queries = [
  {
    "question": "What is the accession number in uniprot of the human gene LCT (filter on rdfs:label)? Return only unique protein URIs",
    "endpoint": "https://sparql.uniprot.org/sparql/",
    "query": """PREFIX up: <http://purl.uniprot.org/core/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
SELECT DISTINCT ?protein
WHERE{
    ?protein a up:Protein .
    ?protein up:organism taxon:9606 .
    ?protein up:encodedBy ?gene .
    ?gene skos:prefLabel "LCT" .
}""",
  },
  {
    # NOTE: The "mature" part in the question makes it harder to answer
    # "question": "How do I filter for reviewed (mouse) proteins whose mature form carries an N-terminal glycine? Return protein URI and AA sequence",
    "question": "How do I filter for reviewed mouse proteins which carry an N-terminal glycine? Return protein URI and AA sequence",
    "endpoint": "https://sparql.uniprot.org/sparql/",
    "query": """PREFIX up: <http://purl.uniprot.org/core/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?protein ?sequence
WHERE
{
    ?protein a up:Protein ;
        up:organism taxon:10090 ;  # Taxonomy ID for Mus musculus (Mouse)
        up:reviewed true ;
        up:sequence ?isoform .
    ?isoform rdf:value ?sequence .
    # Ensure the N-terminal amino acid is Glycine (G)
    FILTER (STRSTARTS(?sequence, "G"))
}""",
  },

  {
    "question": "How could I download a table that only includes the Rhea reactions for which there is experimental evidence? Return only the rhea URI",
    "endpoint": "https://sparql.uniprot.org/sparql/",
    "query": """PREFIX up: <http://purl.uniprot.org/core/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?rhea
WHERE {
  # ECO 269 is experimental evidence
  BIND (<http://purl.obolibrary.org/obo/ECO_0000269> as ?evidence)
  #GRAPH <http://sparql.uniprot.org/uniprot> {
    ?protein up:reviewed true ;
      up:annotation ?a ;
      up:attribution ?attribution  .

    ?a a up:Catalytic_Activity_Annotation ;
      up:catalyticActivity ?ca .
    ?ca up:catalyzedReaction ?rhea .

    [] rdf:subject ?a ;
      rdf:predicate up:catalyticActivity ;
      rdf:object ?ca ;
      up:attribution ?attribution .

    ?attribution up:evidence ?evidence .
  #}
}""",
  },

  {
    "question": "Which human proteins are enzymes catalyzing a reaction involving sterols? Return the protein, sterol and reaction URI",
    "endpoint": "https://sparql.uniprot.org/sparql/",
    "query": """PREFIX up: <http://purl.uniprot.org/core/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
SELECT DISTINCT ?protein ?sterol ?reaction
WHERE {
  SERVICE <https://sparql.rhea-db.org/sparql> {
    ?reaction rdfs:subClassOf rh:Reaction .
    ?reaction rh:side/rh:contains/rh:compound ?compound .
    ?compound rh:chebi ?sterol .
    ?sterol rdfs:subClassOf* CHEBI:15889 .
  }
  ?protein a up:Protein ;
    up:organism taxon:9606 ;
    up:annotation/up:catalyticActivity/up:catalyzedReaction ?reaction .
}""",
  },

  {
    "question": "Which are the human proteins associated with cancer (which have cancer in their disease label)? Return the unique disease label, and HGNC symbol",
    "endpoint": "https://sparql.uniprot.org/sparql/",
    "query": """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX up:<http://purl.uniprot.org/core/>
PREFIX taxon:<http://purl.uniprot.org/taxonomy/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX orth:<http://purl.org/net/orth#>
PREFIX dcterms:<http://purl.org/dc/terms/>
PREFIX obo:<http://purl.obolibrary.org/obo/>
PREFIX lscr:<http://purl.org/lscr#>
PREFIX genex:<http://purl.org/genex#>
PREFIX sio: <http://semanticscience.org/resource/>
SELECT DISTINCT ?diseaseLabel ?hgncSymbol
WHERE {
  	?humanProtein a up:Protein ;
      	# up:organism/up:scientificName 'Homo sapiens' ;
        up:organism taxon:9606 ;
      	up:annotation ?annotation ;
      	rdfs:seeAlso ?hgnc .
  	?hgnc up:database <http://purl.uniprot.org/database/HGNC> ;
      	rdfs:comment ?hgncSymbol .
  	?annotation a up:Disease_Annotation ;
      		up:disease ?disease .
  	?disease skos:prefLabel ?diseaseLabel.
  	FILTER CONTAINS (LCASE(?diseaseLabel), "cancer")
}""",
  },


#   {
#     "question": "Which are the human genes associated with cancer (which have cancer in their disease label) and their orthologs expressed in the rat brain? Return the disease label, human gene URI, human gene HGNC symbol, ortholog rat gene URI",
#     "endpoint": "https://sparql.uniprot.org/sparql/",
#     "query": """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
# PREFIX up:<http://purl.uniprot.org/core/>
# PREFIX taxon:<http://purl.uniprot.org/taxonomy/>
# PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
# PREFIX orth:<http://purl.org/net/orth#>
# PREFIX dcterms:<http://purl.org/dc/terms/>
# PREFIX obo:<http://purl.obolibrary.org/obo/>
# PREFIX lscr:<http://purl.org/lscr#>
# PREFIX genex:<http://purl.org/genex#>
# PREFIX sio: <http://semanticscience.org/resource/>
# SELECT DISTINCT ?diseaseLabel ?humanProtein ?hgncSymbol ?orthologRatProtein ?orthologRatGene
# WHERE {
#   SERVICE <https://sparql.uniprot.org/sparql> {
# 	SELECT DISTINCT * WHERE {
#   	?humanProtein a up:Protein ;
#       	up:organism/up:scientificName 'Homo sapiens' ;
#       	up:annotation ?annotation ;
#       	rdfs:seeAlso ?hgnc .
#   	?hgnc up:database <http://purl.uniprot.org/database/HGNC> ;
#       	rdfs:comment ?hgncSymbol .
#   	?annotation a up:Disease_Annotation ;
#       		up:disease ?disease .
#   	?disease skos:prefLabel ?diseaseLabel.
#   	FILTER CONTAINS (?diseaseLabel, "cancer")
# 	}
#   }
#   SERVICE <https://sparql.omabrowser.org/sparql/> {
# 	SELECT ?humanProtein ?orthologRatProtein ?orthologRatGene WHERE {
#   	?humanProteinOma a orth:Protein ;
#       		lscr:xrefUniprot ?humanProtein .
#   	?orthologRatProtein a orth:Protein ;
#       	sio:SIO_010079 ?orthologRatGene ;
#       	orth:organism/obo:RO_0002162/up:scientificName 'Rattus norvegicus' .
#   	?cluster a orth:OrthologsCluster .
#   	?cluster orth:hasHomologousMember ?node1 .
#   	?cluster orth:hasHomologousMember ?node2 .
#   	?node1 orth:hasHomologousMember* ?humanProteinOma .
#   	?node2 orth:hasHomologousMember* ?orthologRatProtein .
#   	FILTER(?node1 != ?node2)
# 	}
#   }
#   SERVICE <https://www.bgee.org/sparql/> {
# 	?orthologRatGene genex:isExpressedIn ?anatEntity ;
#   		orth:organism ?ratOrganism .
# 	?anatEntity rdfs:label 'brain' .
# 	?ratOrganism obo:RO_0002162 taxon:10116 .
#   }
# }""",
#   },

]

def result_sets_are_same(gen_set, ref_set) -> bool:
    gen_set, ref_set = list(gen_set), list(ref_set)
    for item in gen_set:
        if item not in ref_set:
            print(f"Missing from reference: {item}")
            return False
    for item in ref_set:
        if item not in gen_set:
            print(f"Missing from generated: {item}")
            return False
    return True

QLEVER_UNIPROT = "https://qlever.cs.uni-freiburg.de/api/uniprot"

client = OpenAI()
def answer_no_rag(question: str):
    response = client.chat.completions.create(
        model=settings.llm_model,
        messages=[
            {"role": "system", "content": settings.system_prompt},
            {"role": "user", "content": question},
        ],
        stream=False,
        temperature=0,
        #   response_format={ "type": "json_object" },
    )
    return response.choices[0].message.content

def answer_rag_without_validation(question: str):
    response = requests.post("http://localhost:8000/chat",
        json={
            "messages": [{"role": "user", "content": question}],
            "stream": False,
            "api_key": expasy_api_key,
            "validate": False,
        },
        timeout=60,
    )
    return response.json()["choices"][0]["message"]["content"]

def answer_rag_with_validation(question: str):
    response = requests.post("http://localhost:8000/chat",
        json={
            "messages": [{"role": "user", "content": question}],
            "stream": False,
            "api_key": expasy_api_key,
        },
        timeout=60,
    )
    return response.json()["choices"][0]["message"]["content"]


list_of_approaches = [
    answer_no_rag,
    answer_rag_without_validation,
    answer_rag_with_validation,
]

number_of_tries = 3

success = [0, 0, 0]
mismatch = [0, 0, 0]
fail = [0, 0, 0]


print(f"🧪 Testing {len(example_queries)} queries")
for i, test_query in enumerate(example_queries):
    # if i < 1:
    #     continue
    # Execute the reference query
    ref_endpoint = SPARQLWrapper(test_query["endpoint"])
    # ref_endpoint = SPARQLWrapper(QLEVER_UNIPROT)
    ref_endpoint.setReturnFormat(JSON)
    # ref_endpoint.setTimeout(200)
    ref_endpoint.setQuery(test_query["query"])
    results = ref_endpoint.query().convert()
    res_from_ref = results["results"]["bindings"]


    for approach_index, approach in enumerate(list_of_approaches):
        for t in range(number_of_tries):
            # resp = requests.post("http://localhost:8000/chat",
            #     json={
            #         "messages": [{"role": "user", "content": test_query["question"]}],
            #         "stream": False,
            #         "api_key": expasy_api_key,
            #     },
            #     timeout=60,
            # )
            # chat_resp_md = resp.json()["choices"][0]["message"]["content"]

            chat_resp_md = approach(test_query["question"])

            try:
                generated_sparql = extract_sparql_queries(chat_resp_md)[-1]
                if generated_sparql["query"].strip() == test_query["query"].strip():
                    print(f"✅ {t+1}/{number_of_tries} {test_query['question']}. EXACT MATCH")
                    success[approach_index] += 1
                    continue

                # Execute the generated query
                sparql_endpoint = SPARQLWrapper(generated_sparql["endpoint"])
                # sparql_endpoint = SPARQLWrapper(QLEVER_UNIPROT)
                sparql_endpoint.setReturnFormat(JSON)
                sparql_endpoint.setTimeout(200)
                sparql_endpoint.setQuery(generated_sparql["query"])
                results = sparql_endpoint.query().convert()
                res_from_generated = results["results"]["bindings"]

                if not result_sets_are_same(res_from_generated, res_from_ref):
                    mismatch[approach_index] += 1
                    raise Exception(f"\nResults mismatch. Ref: {len(res_from_ref)} != gen: {len(res_from_generated)}\n")
                else:
                    print(f"✅ {t+1}/{number_of_tries} {test_query['question']} = {len(res_from_generated)}")
                    success[approach_index] += 1

            except Exception as e:
                fail[approach_index] += 1
                print(f"❌ {t+1}/{number_of_tries} {test_query['question']} {e}. Generated query:")
                print(generated_sparql["query"])
                print("Correct query:")
                print(test_query["query"])
                print("")
            # print(f"Results: {len(results['results']['bindings'])}")

    print(f"⚖️ No RAG - Success: {success[0]}, Mismatch: {mismatch[0]}, Error: {fail[0]-mismatch[0]}")
    print(f"⚖️ RAG without validation - Success: {success[1]}, Mismatch: {mismatch[1]}, Error: {fail[1]-mismatch[1]}")
    print(f"⚖️ RAG with validation - Success: {success[2]}, Mismatch: {mismatch[2]}, Error: {fail[2]-mismatch[2]}")

/home/vemonet/dev/expasy/expasy-chat/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🧪 Testing 5 queries
❌ 1/3 What is the accession number in uniprot of the human gene LCT (filter on rdfs:label)? Return only unique protein URIs QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'<!DOCTYPE html SYSTEM "about:legacy-compat">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head><title>UniProt</title><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"/><link href="/" rel="home"/><link href="/base.css" type="text/css" rel="stylesheet"/><link type="image/vnd.microsoft.icon" href="/uniprot-favicon.ico" rel="shortcut icon"/><link href="/uniprot-sparql.css" type="text/css" rel="stylesheet"/><link href="http://creativecommons.org/licenses/by/4.0/" rel="license"/><script src="https://cdn.jsdelivr.net/npm/yasgui-yasqe@2.11.22/dist/yasqe.bundled.min.js"></script><link type="text/css" rel="stylesheet" href="https://cdn.jsdelivr.net/yasqe/2.2/yasqe.min.css"/><meta content="width=dev

KeyboardInterrupt: 